In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import seaborn as sns
from pathlib import Path

In [9]:
table_04=pd.read_excel("2020.04-2020.07(fabric width for HKU).xlsx")
table_07=pd.read_excel("2020.07-2020.10(fabric width for HKU).xlsx")
table_10=pd.read_excel("2020.10-2021.01(fabric width for HKU).xlsx")
table_01=pd.read_excel("2021.01-2021.04(fabric width for HKU).xlsx")
table_all=pd.concat([table_04,table_07,table_10,table_01], axis=0)
table_all.head()

,ETD-出货日期,GF_NO-品名,job_no-排单号,fabric_no-布号,fnsend_no-后整送布单,Quantity-卷长,design_width-工艺门幅,Actual_width-实际门幅,Fabric_Type-布种代码,Warp_Density-经密,...,SUM_Reeding-入筘数,reed_width-筘幅,ReedNo_type-筘号整理类别,Finish_list-整理方式,Mercerizing-丝光,Wash_Method-水洗方式,FabricNumber-织物组织类别,Color_Type-是否本白,Is_Piece_Dye-是否CPB,Shrinkage-纬向缩水标准(分档前)
0,2020-04,DC4040-19D898KMY,PT202189,Q21112220001,FA2003190106,94.8,58.5,58.5,D,165.0,...,2#,69.46,B,21+31+171+,18BE,NW,15,N,0,-2.0
1,2020-04,DC4040-19D898KMY,PT202189,Q21112220002,FA2003190106,93.0,58.5,58.8,D,165.0,...,2#,69.46,B,21+31+171+,18BE,NW,15,N,0,-2.0
2,2020-04,DC4040-19D898KMY,PT202189,Q21112220004,FA2003190106,55.5,58.5,58.5,D,165.0,...,2#,69.46,B,21+31+171+,18BE,NW,15,N,0,-2.0
3,2020-04,DC4040-19D898KMY,PT202189,Q21112220005,FA2003190106,94.8,58.5,59.0,D,165.0,...,2#,69.46,B,21+31+171+,18BE,NW,15,N,0,-2.0
4,2020-04,DC4040-19D898KMY,PT202189,Q21112220006,FA2003190106,75.0,58.5,58.8,D,165.0,...,2#,69.46,B,21+31+171+,18BE,NW,15,N,0,-2.0


In [10]:
drop_cols1=['ETD-出货日期','GF_NO-品名','job_no-排单号','fabric_no-布号','fnsend_no-后整送布单','Quantity-卷长']
table_train_X=table_all.drop(drop_cols1,axis=1)
drop_cols_prod=['NO']
table_prod_X=pd.read_excel("Data+Test+of+fabric+width+for+HKU.xlsx").drop(drop_cols_prod,axis=1)
raw_data=pd.concat([table_train_X,table_prod_X], axis=0)
raw_data.head()

,design_width-工艺门幅,Actual_width-实际门幅,Fabric_Type-布种代码,Warp_Density-经密,Weft_Density-纬密,F_Actual_count-纬向纱支,W_Actual_count-经向纱支,Component-成分,Reed_NO-筘号,SUM_Reeding-入筘数,reed_width-筘幅,ReedNo_type-筘号整理类别,Finish_list-整理方式,Mercerizing-丝光,Wash_Method-水洗方式,FabricNumber-织物组织类别,Color_Type-是否本白,Is_Piece_Dye-是否CPB,Shrinkage-纬向缩水标准(分档前)
0,58.5,58.5,D,165.0,95.0,40,40,100.0% CTN,140.0,2#,69.46,B,21+31+171+,18BE,NW,15,N,0,-2.0
1,58.5,58.8,D,165.0,95.0,40,40,100.0% CTN,140.0,2#,69.46,B,21+31+171+,18BE,NW,15,N,0,-2.0
2,58.5,58.5,D,165.0,95.0,40,40,100.0% CTN,140.0,2#,69.46,B,21+31+171+,18BE,NW,15,N,0,-2.0
3,58.5,59.0,D,165.0,95.0,40,40,100.0% CTN,140.0,2#,69.46,B,21+31+171+,18BE,NW,15,N,0,-2.0
4,58.5,58.8,D,165.0,95.0,40,40,100.0% CTN,140.0,2#,69.46,B,21+31+171+,18BE,NW,15,N,0,-2.0


In [11]:
table_train_X['discrepency'] = table_train_X['design_width-工艺门幅'] - table_train_X['Actual_width-实际门幅']

In [12]:
drop_cols2=['Actual_width-实际门幅']
table_train_X_LM=table_train_X.drop(drop_cols2,axis=1)

In [17]:
table_train_Y_dependent=table_train_X['discrepency'].copy()

In [18]:
table_train_Y_dependent.head()

0    0.0
1   -0.3
2    0.0
3   -0.5
4   -0.3
Name: discrepency, dtype: float64

In [16]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
sc_X = StandardScaler()

In [ ]:
SourceData_test_independent=table_prod_X.drop(['Actual_width-实际门幅'], axis=1)
SourceData_test_dependent=table_prod_X["Actual_width-实际门幅"].copy()

In [ ]:
X_train=sc_X.fit_transform(table_train_X_LM) #scale the independent variables
y_train=table_train_Y_dependent # scaling is not required for dependent variable
X_test=sc_X.transform(SourceData_test_independent)
y_test=SourceData_test_dependent